In [1]:
"""
Create EOW Services for Web Mapping
"""

# Map Services Extent
ext_shp = '/home/jasp/mrgis/flainar_serv/lmt_pt.shp'

# Tiles width and height
width  = 50000
height = 50000

# Output EPSG
oepsg = 3763

# Layers Definition
layers = {
    'gsrv_osm_polygons' : (
        "SELECT polygonid, ST_Transform(ST_MakeValid(geom), 3763) AS geom FROM ("
            "SELECT op.polygonid, op.geom, "
            "array_agg(osmcat.key) AS okey, array_agg(osmcat.value) AS ovalue "
            "FROM osm_polygons AS op "
            "INNER JOIN polygons_osmcat AS ocat "
            "ON op.polygonid = ocat.polygoncatid "
            "INNER JOIN osmcategory AS osmcat "
            "ON ocat.osmcategoryid = osmcat.osmcategoryid "
            "GROUP BY op.polygonid, op.geom"
        ") AS foo "
        "WHERE NOT('boundary' = ANY(ovalue))"
    ),
    'gsrv_osm_points' : (
        "SELECT positionid, name, ST_Transform(geom, 3763) AS geom "
        "FROM osm_position "
        "WHERE osm_id IS NOT NULL"
    )
}

simplification = {'01' : 0.01, '05' : 0.05, '1' : 0.1}

simplayers = ['gsrv_osm_polygons']

geomcols = {'gsrv_osm_polygons' : 'geom', 'gsrv_osm_points' : 'geom'}

# RefGRID name
refgrid = 'refgrid'

# DB Name
db = 'flainar_gh'

# GeoServer data
ws = 'eowdeploy'
store = 'ghost'

In [2]:
import os
from shapely.wkt import loads
from glass.geo.gt.sample import create_fishnet
from glass.geo.gt.fmshp import shp_to_obj
from glass.sql.to import df_to_db
from glass.geo.gql.sply import geom_simplification
from glass.sql.i import cols_name
from glass.sql.to import q_to_ntbl
from glass.sql.fm import q_to_obj
from glass.web.geosrv.ws import create_ws
from glass.web.geosrv.stores import create_pgstore
from glass.web.geosrv.lyrs import pub_pglyr

In [3]:
from glass.sql.tbl import del_tables

for l in layers:
    del_tables(db, l, isBasename=True)

In [4]:
# Create reference GRID
refgrid_shp = create_fishnet(ext_shp, os.path.join(
    os.path.dirname(ext_shp),
    refgrid + '.shp'
), width, height)

# Remove cells not intersecting with ext_shp
df_grid = shp_to_obj(refgrid_shp, srs_to=oepsg)
df_ref  = shp_to_obj(ext_shp, srs_to=oepsg)

geom = loads(df_ref.loc[0, 'geometry'].wkt)

df_grid['is_int'] = df_grid.geometry.intersects(geom)
df_grid = df_grid[df_grid.is_int == True]

df_grid.reset_index(inplace=True)

df_grid.drop(['index', 'FID', 'is_int'], axis=1, inplace=True)

df_grid['cell_id'] = df_grid.index + 1

# Reference GRID to PostgreSQL
refgrid = df_to_db(db, df_grid, refgrid, colGeom='geometry', epsg=oepsg, geomType='Polygon')

/home/jasp/.virtualenvs/gaspenv/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/jasp/.virtualenvs/gaspenv/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [5]:
# Produce layer tables
for l in layers:
    q_to_ntbl(db, l, layers[l], api='psql')

In [6]:
# Geometries simplification

simplyres = {}
lyrcols   = {}

for l in layers:
    lyrcols[l]   = cols_name(db, l)
    lyrcols[l].remove(geomcols[l])

for l in simplayers:
    simplyres[l] = []
    
    for s in simplification:
        otbl = geom_simplification(
            db, l, lyrcols[l], geomcols[l],
            simplification[s], l + '_' + s
        )
        
        simplyres[l].append(otbl)

print(simplyres)

{'gsrv_osm_polygons': ['gsrv_osm_polygons_01', 'gsrv_osm_polygons_05', 'gsrv_osm_polygons_1']}


In [7]:
# Create workspace and store in GeoServer

create_ws(ws, overwrite=True)

create_pgstore(store, ws, db)

<Response [201]>

In [8]:
# Clip Geometries

refgrid = 'refgrid'

df_grid = q_to_obj(db, refgrid, geomCol='geom', epsg=oepsg)

Q = (
    "SELECT {c}, ST_Intersection({g}, ST_GeomFromText('{w}', {ep})) AS {g} "
    "FROM {t} "
    "WHERE ST_Intersects({g}, ST_GeomFromText('{w}', {ep}))"
)

# Publish layers with all data
for l in layers:
    pub_pglyr(ws, store, l, title='tt_' + l)
    
    if l in simplyres:
        for t in simplyres[l]:
            pub_pglyr(ws, store, t, title='tt_' + t)

for idx, row in df_grid.iterrows():
    # For Layer in layers
    for l in layers:
        gc = geomcols[l]
        
        # Clip
        nt = q_to_ntbl(db, l + '_id' + str(row.cell_id), Q.format(
            c=", ".join(lyrcols[l]), g=gc, w=row.geom.wkt,
            ep=str(oepsg), t=l
        ), api='psql')
        
        # Publish layer in GeoServer
        pub_pglyr(ws, store, nt, title='tt_' + nt)
        
        # If there is some simplified tables
        if l in simplyres:
            # Clip simplied geometries
            for t in simplyres[l]:
                _nt = q_to_ntbl(db, t + '_id' + str(row.cell_id), Q.format(
                    c=", ".join(lyrcols[l]), g=gc, w=row.geom.wkt,
                    ep=str(oepsg), t=t
                ), api='psql')
                
                pub_pglyr(ws, store, _nt, title='tt_' + _nt)

In [ ]:
for t in simplyres[l]:
    print(t)

In [ ]:
base = df_ref.plot(color='white', edgecolor='black')

df_grid.plot(ax=base, color=None, edgecolor='red')

In [ ]:
print(df_grid.geometry.wkt)